# Medicare Part D - Prescription ETL
---
- placeholder text for description
---
### Dependencies and Setup

In [1]:
# For extraction and transformation
import os
import pandas as pd
from pprint import pprint
from bs4 import BeautifulSoup
import requests

# For loading
from sqlalchemy import create_engine
from sqlalchemy.orm import Session
from datetime import datetime as dt


---
## Extraction
- CMS Part D Prescriber Summary Reports

In [2]:
# load file
cms_path = os.path.join('.', 'Raw', 'part-d-prescriber-national-summary-report-calendar-year-2016.csv')
prescriber_df = pd.read_csv(cms_path)

# preview data
prescriber_df.head()

,Drug Name,Generic Name,Number of Prescribers,Number of Medicare Part D Claims,Number of Standardized 30-Day Part D Fills,Aggregate Cost Paid for Part D Claims,Number of Medicare Beneficiaries,GE65 Suppression Flag,Number of Medicare Part D Claims for Beneficiaries 65+,Number of Standardized 30-Day Part D Fills for Beneficiaries 65+,Aggregate Cost Paid for Part D Claims for Beneficiaries 65+,Beneficiary 65+ Suppression Flag,Number of Medicare Beneficiaries 65+,Aggregate Cost Share for Beneficiaries with Low Income Subsidy,Aggregate Cost Share for Beneficiaries with No Low Income Subsidy,Opioid Drug Flag,Extended-Release Opioid Drug Flag,Antibiotic Drug Flag,Antipsychotic Drug Flag
0,1ST TIER UNIFINE PENTIPS,"PEN NEEDLE, DIABETIC",2832.0,8913,14253.8,216273.87,3428.0,NaN,6665.0,10961.8,160300.93,NaN,2599.0,9209.09,44253.44,N,N,N,N
1,1ST TIER UNIFINE PENTIPS PLUS,"PEN NEEDLE, DIABETIC",5048.0,14610,25998.3,378040.97,6564.0,NaN,11088.0,20230.7,285670.63,NaN,5023.0,12805.19,83730.89,N,N,N,N
2,ABACAVIR,ABACAVIR SULFATE,5650.0,66707,73303.6,19384433.39,9007.0,NaN,22814.0,26105.0,6750386.75,NaN,3034.0,41654.01,406354.27,N,N,N,N
3,ABACAVIR-LAMIVUDINE,ABACAVIR SULFATE/LAMIVUDINE,3257.0,13903,14875.2,14752760.25,6629.0,NaN,4538.0,5060.3,4949203.56,NaN,2184.0,2751.36,92745.45,N,N,N,N
4,ABACAVIR-LAMIVUDINE-ZIDOVUDINE,ABACAVIR/LAMIVUDINE/ZIDOVUDINE,1270.0,9768,10378.4,12787721.57,1333.0,NaN,3463.0,3835.8,4681177.83,NaN,480.0,21947.69,324389.79,N,N,N,N


In [3]:
# inspect column names
pprint(prescriber_df.columns)

Index(['Drug Name', 'Generic Name', ' Number of Prescribers ',
       ' Number of Medicare Part D Claims ',
       ' Number of Standardized 30-Day Part D Fills ',
       'Aggregate Cost Paid for Part D Claims',
       ' Number of Medicare Beneficiaries ', ' GE65 Suppression Flag ',
       ' Number of Medicare Part D Claims for Beneficiaries 65+ ',
       ' Number of Standardized 30-Day Part D Fills for Beneficiaries 65+ ',
       'Aggregate Cost Paid for Part D Claims for Beneficiaries 65+',
       ' Beneficiary 65+ Suppression Flag ',
       ' Number of Medicare Beneficiaries 65+ ',
       'Aggregate Cost Share for Beneficiaries with Low Income Subsidy',
       'Aggregate Cost Share for Beneficiaries with No Low Income Subsidy',
       ' Opioid Drug Flag ', 'Extended-Release Opioid Drug Flag',
       ' Antibiotic Drug Flag ', ' Antipsychotic Drug Flag '],
      dtype='object')


- Medicaid.gov - National Average Drug Acquisition Cost

In [4]:
# load file
nadac_filepath = os.path.join(".","Raw","NADAC__National_Average_Drug_Acquisition_Cost_.csv")
nadac_df = pd.read_csv(nadac_filepath)

# preview table
nadac_df.head()

,NDC Description,NDC,NADAC_Per_Unit,Effective_Date,Pricing_Unit,Pharmacy_Type_Indicator,OTC,Explanation_Code,Classification_for_Rate_Setting,Corresponding_Generic_Drug_NADAC_Per_Unit,Corresponding_Generic_Drug_Effective_Date,As of Date
0,SIMVASTATIN 80 MG TABLET,16714068502,0.08008,03/23/2016,EA,C/I,N,1,G,NaN,NaN,04/13/2016
1,CARVEDILOL 12.5 MG TABLET,68382009405,0.03281,03/23/2016,EA,C/I,N,1,G,NaN,NaN,04/13/2016
2,CLONAZEPAM 1 MG TABLET,93083310,0.02619,03/23/2016,EA,C/I,N,1,G,NaN,NaN,04/13/2016
3,NAPHCON-A EYE DROPS,65008542,0.56743,12/23/2015,ML,C/I,Y,"4, 5, 6",B,NaN,NaN,04/13/2016
4,OFLOXACIN 0.3% EAR DROPS,24208041005,17.27867,02/17/2016,ML,C/I,N,"4, 5",G,NaN,NaN,04/13/2016


- Q1Medicare

In [5]:
# website for retrieval
q1_url = 'https://q1medicare.com/PartD-The-MedicarePartDOutlookAllYears.php'

In [6]:
# get html
q1_html = pd.read_html(q1_url)
type(q1_html)

list

In [7]:
# inspect html
q1_html

[                                                   0   \
 0   Medicare Part D Benefit Parameters for Defined...   
 1          Part D Standard Benefit Design Parameters:   
 2   Deductible - After the Deductible is met, Bene...   
 3   Initial Coverage Limit - Coverage Gap (Donut H...   
 4   Out-of-Pocket Threshold - This is the Total Ou...   
 5   Total Covered Part D Drug Out-of-Pocket Spendi...   
 6   Total Estimated Covered Part D Drug Out-of-Poc...   
 7                      Catastrophic Coverage Benefit:   
 8             Generic/Preferred Multi-Source Drug (3)   
 9                                     Other Drugs (3)   
 10  Part D Full Benefit Dual Eligible (FBDE) Param...   
 11                                       • Deductible   
 12   • Copayments for Institutionalized Beneficiaries   
 13  Maximum Copayments for Non-Institutionalized B...   
 14                              Up to or at 100% FPL:   
 15                    • Up to Out-of-Pocket Threshold   
 16           

In [8]:
# select desired table for df and set labels
q1_raw_df = q1_html[0]
q1_raw_df.columns = ['medicare', '2019', '2018','2017','2016','2015','2014','2013','2012','2011','2010','2009','2008','2007','2006']
q1_raw_df.head(6)

,medicare,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006
0,Medicare Part D Benefit Parameters for Defined...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Part D Standard Benefit Design Parameters:,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006
2,"Deductible - After the Deductible is met, Bene...",$415,$405,$400,$360,$320,$310,$325,$320,$310,$310,$295,$275,$265,$250
3,Initial Coverage Limit - Coverage Gap (Donut H...,"$3,820","$3,750","$3,700","$3,310","$2,960","$2,850","$2,970","$2,930","$2,840","$2,830","$2,700","$2,510","$2,400","$2,250"
4,Out-of-Pocket Threshold - This is the Total Ou...,"$5,100","$5,000","$4,950","$4,850","$4,700","$4,550","$4,750","$4,700","$4,550","$4,550","$4,350","$4,050","$3,850","$3,600"
5,Total Covered Part D Drug Out-of-Pocket Spendi...,"$7,653.75 (1)","$7,508.75 (1)","$7,425.00 (1)","$7,062.50 (1)","$6,680.00 (1)","$6,455.00 (1)","$6,733.75 (1)","$6,657.50 (1)","$6,447.50 (1)","$6,440.00plus a$250 rebate","$6,153.75","$5,726.25","$5,451.25","$5,100.00"


---
## Transformation
- CMS Clean-up:

In [9]:
# eliminate unnecessary spaces in 
red_col_names = [col.strip() for col in prescriber_df.columns]
prescriber_df.columns = red_col_names
pprint(prescriber_df.columns)

Index(['Drug Name', 'Generic Name', 'Number of Prescribers',
       'Number of Medicare Part D Claims',
       'Number of Standardized 30-Day Part D Fills',
       'Aggregate Cost Paid for Part D Claims',
       'Number of Medicare Beneficiaries', 'GE65 Suppression Flag',
       'Number of Medicare Part D Claims for Beneficiaries 65+',
       'Number of Standardized 30-Day Part D Fills for Beneficiaries 65+',
       'Aggregate Cost Paid for Part D Claims for Beneficiaries 65+',
       'Beneficiary 65+ Suppression Flag',
       'Number of Medicare Beneficiaries 65+',
       'Aggregate Cost Share for Beneficiaries with Low Income Subsidy',
       'Aggregate Cost Share for Beneficiaries with No Low Income Subsidy',
       'Opioid Drug Flag', 'Extended-Release Opioid Drug Flag',
       'Antibiotic Drug Flag', 'Antipsychotic Drug Flag'],
      dtype='object')


In [10]:
# look at values in GE65 Suppression Flag Column
prescriber_df['GE65 Suppression Flag'].value_counts()

#    160
*    151
Name: GE65 Suppression Flag, dtype: int64

In [11]:
# look at values in a opiod drug flag column
prescriber_df['Opioid Drug Flag'].value_counts()

N    3301
Y      92
Name: Opioid Drug Flag, dtype: int64

In [12]:
# look at values in antibiotic drug flag column
prescriber_df['Antibiotic Drug Flag'].value_counts()

N    3215
Y     178
Name: Antibiotic Drug Flag, dtype: int64

In [13]:
# turn flag columns into booleans
prescriber_df.loc[:, 'Opioid Drug Flag'] = [(True if flag == 'Y' else False) for flag in prescriber_df.loc[:, 'Opioid Drug Flag']]
prescriber_df.loc[:, 'Extended-Release Opioid Drug Flag'] = [(True if flag == 'Y' else False) for flag in prescriber_df.loc[:, 'Extended-Release Opioid Drug Flag']]
prescriber_df.loc[:, 'Antibiotic Drug Flag'] = [(True if flag == 'Y' else False) for flag in prescriber_df.loc[:, 'Antibiotic Drug Flag']]
prescriber_df.loc[:, 'Antipsychotic Drug Flag'] = [(True if flag == 'Y' else False) for flag in prescriber_df.loc[:, 'Antipsychotic Drug Flag']]

In [14]:
# drop unnecessary columns
final_prescriber_df = prescriber_df.drop(['Beneficiary 65+ Suppression Flag', 'GE65 Suppression Flag'], axis = 1)

In [15]:
# calculate average cost per fill
final_prescriber_df['average_cost_per_fill'] = final_prescriber_df.loc[:, 'Aggregate Cost Paid for Part D Claims'].divide(final_prescriber_df.loc[:, 'Number of Standardized 30-Day Part D Fills'])

In [16]:
# calculate average annual cost of each drug (assume 12 fills per year)
final_prescriber_df['average_yearly_cost'] = final_prescriber_df.loc[:, 'average_cost_per_fill'].multiply(12)

In [17]:
# preview dataframe
final_prescriber_df.head()

,Drug Name,Generic Name,Number of Prescribers,Number of Medicare Part D Claims,Number of Standardized 30-Day Part D Fills,Aggregate Cost Paid for Part D Claims,Number of Medicare Beneficiaries,Number of Medicare Part D Claims for Beneficiaries 65+,Number of Standardized 30-Day Part D Fills for Beneficiaries 65+,Aggregate Cost Paid for Part D Claims for Beneficiaries 65+,Number of Medicare Beneficiaries 65+,Aggregate Cost Share for Beneficiaries with Low Income Subsidy,Aggregate Cost Share for Beneficiaries with No Low Income Subsidy,Opioid Drug Flag,Extended-Release Opioid Drug Flag,Antibiotic Drug Flag,Antipsychotic Drug Flag,average_cost_per_fill,average_yearly_cost
0,1ST TIER UNIFINE PENTIPS,"PEN NEEDLE, DIABETIC",2832.0,8913,14253.8,216273.87,3428.0,6665.0,10961.8,160300.93,2599.0,9209.09,44253.44,False,False,False,False,15.173068,182.076810
1,1ST TIER UNIFINE PENTIPS PLUS,"PEN NEEDLE, DIABETIC",5048.0,14610,25998.3,378040.97,6564.0,11088.0,20230.7,285670.63,5023.0,12805.19,83730.89,False,False,False,False,14.540988,174.491857
2,ABACAVIR,ABACAVIR SULFATE,5650.0,66707,73303.6,19384433.39,9007.0,22814.0,26105.0,6750386.75,3034.0,41654.01,406354.27,False,False,False,False,264.440401,3173.284814
3,ABACAVIR-LAMIVUDINE,ABACAVIR SULFATE/LAMIVUDINE,3257.0,13903,14875.2,14752760.25,6629.0,4538.0,5060.3,4949203.56,2184.0,2751.36,92745.45,False,False,False,False,991.768867,11901.226404
4,ABACAVIR-LAMIVUDINE-ZIDOVUDINE,ABACAVIR/LAMIVUDINE/ZIDOVUDINE,1270.0,9768,10378.4,12787721.57,1333.0,3463.0,3835.8,4681177.83,480.0,21947.69,324389.79,False,False,False,False,1232.147688,14785.772262


- Medicaid NADAC Clean-up:

In [18]:
# view column labels
nadac_df.columns

Index(['NDC Description', 'NDC', 'NADAC_Per_Unit', 'Effective_Date',
       'Pricing_Unit', 'Pharmacy_Type_Indicator', 'OTC', 'Explanation_Code',
       'Classification_for_Rate_Setting',
       'Corresponding_Generic_Drug_NADAC_Per_Unit',
       'Corresponding_Generic_Drug_Effective_Date', 'As of Date'],
      dtype='object')

In [19]:
# drop unnecessary columns
to_drop = ["NDC","Pharmacy_Type_Indicator","Explanation_Code","Corresponding_Generic_Drug_NADAC_Per_Unit","Corresponding_Generic_Drug_Effective_Date","As of Date"]
nadac_df.drop(to_drop, inplace=True, axis=1)

# preview change
nadac_df.head()

,NDC Description,NADAC_Per_Unit,Effective_Date,Pricing_Unit,OTC,Classification_for_Rate_Setting
0,SIMVASTATIN 80 MG TABLET,0.08008,03/23/2016,EA,N,G
1,CARVEDILOL 12.5 MG TABLET,0.03281,03/23/2016,EA,N,G
2,CLONAZEPAM 1 MG TABLET,0.02619,03/23/2016,EA,N,G
3,NAPHCON-A EYE DROPS,0.56743,12/23/2015,ML,Y,B
4,OFLOXACIN 0.3% EAR DROPS,17.27867,02/17/2016,ML,N,G


In [20]:
# change date to a sort-convinient format
nadac_df['Effective_Date'] = [dt.strftime(dt.strptime(i, "%m/%d/%Y"), "%y%m%d") for i in nadac_df['Effective_Date']]

# sort by name, then effective date in descending order
sort_by = ['NDC Description', 'Effective_Date']
nadac_df.sort_values(sort_by, inplace=True)

# preview change
nadac_df.head()

,NDC Description,NADAC_Per_Unit,Effective_Date,Pricing_Unit,OTC,Classification_for_Rate_Setting
1057615,12-HR DECONGEST 120 MG CAPLET,0.33159,131121,EA,Y,G
1177205,12-HR DECONGEST 120 MG CAPLET,0.33159,131121,EA,Y,G
1180481,12-HR DECONGEST 120 MG CAPLET,0.33159,131121,EA,Y,G
1448137,12-HR DECONGEST 120 MG CAPLET,0.33159,131121,EA,Y,G
1499771,12-HR DECONGEST 120 MG CAPLET,0.33159,131121,EA,Y,G


In [43]:
# Function to split the NDC Description into its name and dosage
def dose_split(rx):
    parse = rx.split()
    found = False
    name = parse[0]
    parse.pop(0)
    dosage = ""
    
    for i in parse:
        if (found):
            dosage = dosage + " " + i
        elif (i[0].isdigit() or i == "TABLET"):
            found = True
            dosage = i
        else:
            name = name + " " + i
    return name, dosage

# Test run on last entry in table
dose_split('ZZZQUIL 50 MG/30 ML LIQUID')

('ZZZQUIL', '50 MG/30 ML LIQUID')

In [44]:
# Create 2 new columns for the name and dosage to help with merging to CMS dataset
names = []
doses = []
for i in nadac_df['NDC Description']:
    name, dose = dose_split(i)
    names.append(name)
    doses.append(dose)
nadac_df['rx_name'] = names
nadac_df['dosage'] = doses
nadac_df.head()

,NDC Description,NADAC_Per_Unit,Effective_Date,Pricing_Unit,OTC,Classification_for_Rate_Setting,rx_name,dosage
1057615,12-HR DECONGEST 120 MG CAPLET,0.33159,131121,EA,Y,G,12-HR DECONGEST,120 MG CAPLET
1177205,12-HR DECONGEST 120 MG CAPLET,0.33159,131121,EA,Y,G,12-HR DECONGEST,120 MG CAPLET
1180481,12-HR DECONGEST 120 MG CAPLET,0.33159,131121,EA,Y,G,12-HR DECONGEST,120 MG CAPLET
1448137,12-HR DECONGEST 120 MG CAPLET,0.33159,131121,EA,Y,G,12-HR DECONGEST,120 MG CAPLET
1499771,12-HR DECONGEST 120 MG CAPLET,0.33159,131121,EA,Y,G,12-HR DECONGEST,120 MG CAPLET


In [45]:
# drop duplicate values of full name keeping only the newest value of the set
reduc_nadac_df = nadac_df.drop_duplicates('NDC Description', keep='last')
reduc_nadac_df.head()

,NDC Description,NADAC_Per_Unit,Effective_Date,Pricing_Unit,OTC,Classification_for_Rate_Setting,rx_name,dosage
6148809,12-HR DECONGEST 120 MG CAPLET,0.32280,181017,EA,Y,G,12-HR DECONGEST,120 MG CAPLET
2983190,12.5CPD-100GFN-30PSE LIQUID,0.09835,131121,ML,Y,G,12.5CPD-100GFN-30PSE LIQUID,
2949453,12.5CPD-1DCPM-30PSE LIQUID,0.09810,150218,ML,Y,G,12.5CPD-1DCPM-30PSE LIQUID,
6127069,12HR NASAL DECONGEST ER 120 MG,0.32280,181017,EA,Y,G,12HR NASAL DECONGEST ER,120 MG
6133647,24H NASAL ALLERGY 55 MCG SPRAY,0.71712,181017,ML,Y,G,24H NASAL ALLERGY,55 MCG SPRAY


In [46]:
reduc_nadac_df['rx_name'].count()

8150

In [47]:
nadac_outpath = os.path.join(".","Set","NADAC_Reduc.csv")
reduc_nadac_df.to_csv(nadac_outpath)

In [24]:
# Q1 Clean-up:
partD_df = q1_raw_df[:7]
partD_df

,medicare,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006
0,Medicare Part D Benefit Parameters for Defined...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Part D Standard Benefit Design Parameters:,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006
2,"Deductible - After the Deductible is met, Bene...",$415,$405,$400,$360,$320,$310,$325,$320,$310,$310,$295,$275,$265,$250
3,Initial Coverage Limit - Coverage Gap (Donut H...,"$3,820","$3,750","$3,700","$3,310","$2,960","$2,850","$2,970","$2,930","$2,840","$2,830","$2,700","$2,510","$2,400","$2,250"
4,Out-of-Pocket Threshold - This is the Total Ou...,"$5,100","$5,000","$4,950","$4,850","$4,700","$4,550","$4,750","$4,700","$4,550","$4,550","$4,350","$4,050","$3,850","$3,600"
5,Total Covered Part D Drug Out-of-Pocket Spendi...,"$7,653.75 (1)","$7,508.75 (1)","$7,425.00 (1)","$7,062.50 (1)","$6,680.00 (1)","$6,455.00 (1)","$6,733.75 (1)","$6,657.50 (1)","$6,447.50 (1)","$6,440.00plus a$250 rebate","$6,153.75","$5,726.25","$5,451.25","$5,100.00"
6,Total Estimated Covered Part D Drug Out-of-Poc...,"$8,139.54plus a 75% brand discount","$8,417.60plus a 65% brand discount","$8,071.16plus a 60% brand discount","$7,515.22plus a 55% brand discount","$7,061.76plus a 55% brand discount","$6,690.77plus a 52.50% brand discount","$6,954.52plus a 52.50% brand discount","$6,730.39plus a 50% brand discount","$6,483.72plus a 50% brand discount",NaN,NaN,NaN,NaN,NaN


In [25]:
df_catastrophic_coverage = q1_raw_df.iloc[8:13,:]
df_catastrophic_coverage

,medicare,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006
8,Generic/Preferred Multi-Source Drug (3),$3.40 (3),$3.35 (3),$3.30 (3),$2.95 (3),$2.65 (3),$2.55 (3),$2.65 (3),$2.60 (3),$2.50 (3),$2.50 (3),$2.40 (3),$2.25 (3),$2.15 (3),$2.00 (3)
9,Other Drugs (3),$8.50 (3),$8.35 (3),$8.25 (3),$7.40 (3),$6.60 (3),$6.35 (3),$6.60 (3),$6.50 (3),$6.30 (3),$6.30 (3),$6.00 (3),$5.60 (3),$5.35 (3),$5.00 (3)
10,Part D Full Benefit Dual Eligible (FBDE) Param...,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006
11,• Deductible,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00
12,• Copayments for Institutionalized Beneficiaries,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00


In [26]:
max_copayments_upto100 = q1_raw_df.iloc[16:19,:]
max_copayments_upto100

,medicare,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006
16,- Generic / Preferred Multi-Source Drug,$1.25,$1.25,$1.20,$1.20,$1.20,$1.20,$1.15,$1.10,$1.10,$1.10,$1.10,$1.05,$1.00,$1.00
17,- Other Drugs,$3.80,$3.70,$3.70,$3.60,$3.60,$3.60,$3.50,$3.30,$3.30,$3.30,$3.20,$3.10,$3.10,$3.00
18,• Above Out-of-Pocket Threshold,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,NaN


In [27]:
max_copayments_over100 = q1_raw_df.iloc[21:24,:]
max_copayments_over100

,medicare,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006
21,- Generic / Preferred Multi-Source Drug,$3.40,$3.35,$3.30,$2.95,$2.65,$2.55,$2.65,$2.60,$2.50,$2.50,$2.40,$2.25,$2.15,$2.00
22,- Other Drugs,$8.50,$8.35,$8.25,$7.40,$6.60,$6.35,$6.60,$6.50,$6.30,$6.30,$6.00,$5.60,$5.35,$5.00
23,• Above Out-of-Pocket Threshold,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00


In [28]:
full_subsidy = q1_raw_df.iloc[26:31,:]
full_subsidy

,medicare,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006
26,• Deductible,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00
27,• Maximum Copayments up to Out-of-Pocket Thres...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28,- Generic / Preferred Multi-Source Drug,$3.40,$3.35,$3.30,$2.95,$2.65,$2.55,$2.65,$2.60,$2.50,$2.50,$2.40,$2.25,$2.15,$2.00
29,- Other Drugs,$8.50,$8.35,$8.25,$7.40,$6.60,$6.35,$6.60,$6.50,$6.30,$6.30,$6.00,$5.60,$5.35,$5.00
30,• Maximum Copay above Out-of-Pocket Threshold,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00


In [29]:
partial_subsidy = q1_raw_df.iloc[33:38,:]
partial_subsidy

,medicare,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006
33,• Deductible,$85.00,$83.00,$82.00,$74.00,$66.00,$63.00,$66.00,$65.00,$63.00,$63.00,$60.00,$56.00,$53.00,$50.00
34,• Coinsurance up to Out-of-Pocket Threshold,15%,15%,15%,15%,15%,15%,15%,15%,15%,15%,15%,15%,15%,15%
35,• Maximum Copayments above Out-of-Pocket Thres...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36,- Generic / Preferred Multi-Source Drug,$3.40,$3.35,$3.30,$2.95,$2.65,$2.55,$2.65,$2.60,$2.50,$2.50,$2.40,$2.25,$2.15,$2.00
37,- Other Drugs,$8.50,$8.35,$8.25,$7.40,$6.60,$6.35,$6.60,$6.50,$6.30,$6.30,$6.00,$5.60,$5.35,$5.00


---
## Loading

In [ ]:
rx_final_df

In [ ]:
engine = create_engine("mysql://root@localhost/partD_rx_db")
connection = engine.connect()
session = Session(bind=engine)

In [ ]:
rx_final_df.to_sql('part_d_rx_db', con=engine, if_exists='replace', index_label='')